In [1]:
import time
import pandas as pd
import numpy as np

CITY_DATA = { 'chicago': 'chicago.csv',
              'new york city': 'new_york_city.csv',
              'washington': 'washington.csv' }

In [11]:
def get_filters():
    
    """
    Asks user to specify a city, month, and day to analyze.

    Returns:
        (str) city - name of the city to analyze
        (str) month - name of the month to filter by, or "all" to apply no month filter
        (str) day - name of the day of week to filter by, or "all" to apply no day filter
    """
    print('Hello! Let\'s explore some US bikeshare data!')
    # get user input for city (chicago, new york city, washington). HINT: Use a while loop to handle invalid inputs
    while True:
            
        try:
            city = input("Would you like to see data for Chicago, New York, or Washington?")
        except Exception as e:
            print(e + " Please try again.")
            #Return to the start of the the loop 
            continue
        
        if city.lower() not in ('chicago', 'new york', 'washington'):
            print(city + " is not a valid city. Please try again")
            continue
        
        else:
            #city was successfully parsed!
            #We're ready to exit the loop.
            break

    city = city.lower()

    
#    while True:
        
#        try:
#            filterBy = input("Would you like to filter the data by month, day, both, or not at all? Type ""none"" for no time filter.")
#
#        except Exception as e:
#            print(e + " Please try again.")
#            #Return to the start of the the loop 
#            continue
        
        
#        if filterBy.lower() not in ('month', 'day', 'both', 'none'):
#            print(filterBy + " is not a valid filter. Please try again")
#            continue
#        else:
            #filter was successfully parsed!
            #We're ready to exit the loop.
#           break
    
#    filterBy = filterBy.lower()


    
#    while filterBy in ('none'):
#        month = "all"
#        day = "all"
#        break

        
    # get user input for month (all, january, february, ... , june)


#    while filterBy in ('month', 'both'):
    while True:        
        try:
            month = input("Which month? Jan, Feb, Mar, Apr, May, or Jun?")
        except Exception as e:
            print(e + " Please try again.")
            #Return to the start of the the loop 
            continue
            
        if month.lower() not in ('jan', 'feb', 'mar', 'apr', 'may', 'jun'):
            print(month + " is not a valid month. Please try again")
            continue
        else:
            #month was successfully parsed!
            #We're ready to exit the loop.
            break           

    month = month.lower()        



    # get user input for day of week (all, monday, tuesday, ... sunday)

#    while filterBy in ('day', 'both'):       
    while True:        

        try:
            day = input("Which day? Please type your response as an integer (e.g., 1=Sunday).")
        except Exception as e:
            print(e + " Please try again.")
            #Return to the start of the the loop 
            continue
            
        if 7 > int(day) < 0:
            print(day + " is not a valid day. Please try again")
            continue
        else:
            #month was successfully parsed!
            #We're ready to exit the loop.
            break   
    
    day = day

    print('-'*40)
    return city, month, day

In [18]:
def load_data(city, month, day):    
    """
    Loads data for the specified city and filters by month and day if applicable.

    Args:
        (str) city - name of the city to analyze
        (str) month - name of the month to filter by, or "all" to apply no month filter
        (str) day - name of the day of week to filter by, or "all" to apply no day filter
    Returns:
        df - Pandas DataFrame containing city data filtered by month and day
    """ 
    
    # load data file into a dataframe
    df = pd.read_csv(CITY_DATA[city])

    # convert the Start Time column to datetime
    df['Start Time'] = pd.to_datetime(df['Start Time'])

    # extract month and day of week from Start Time to create new columns
    df['month'] = df['Start Time'].dt.month
    df['day_of_week'] = df['Start Time'].dt.weekday_name

    # filter by month if applicable
    if month != 'all':
        # use the index of the months list to get the corresponding int
        months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun']
        month = months.index(month) + 1

        # filter by month to create the new dataframe
        df = df[df['month'] == month]

    # filter by day of week if applicable
    if day != 'all':
        # filter by day of week to create the new dataframe
        df = df[df['day_of_week'] == day.title()] 

    return df

In [17]:
def time_stats(df):
    """Displays statistics on the most frequent times of travel."""

    print('\nCalculating The Most Frequent Times of Travel...\n')
    start_time = time.time()

    # display the most common month


    # display the most common day of week


    # display the most common start hour

    # convert the Start Time column to datetime
    df['Start Time'] = pd.to_datetime(df['Start Time'])

    # extract hour from the Start Time column to create an hour column
    df['hour'] = df['Start Time'].dt.hour

    # find the most popular hour
    popular_hour = df['hour'].mode()[0]

    print('Most Popular Start Hour:', popular_hour)

    
    

    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)

In [5]:
def station_stats(df):
    """Displays statistics on the most popular stations and trip."""

    print('\nCalculating The Most Popular Stations and Trip...\n')
    start_time = time.time()

    # display most commonly used start station


    # display most commonly used end station


    # display most frequent combination of start station and end station trip


    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)

    

In [6]:
def trip_duration_stats(df):
    """Displays statistics on the total and average trip duration."""

    print('\nCalculating Trip Duration...\n')
    start_time = time.time()

    # display total travel time


    # display mean travel time


    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)


In [7]:
def user_stats(df):
    """Displays statistics on bikeshare users."""

    print('\nCalculating User Stats...\n')
    start_time = time.time()

    # Display counts of user types


    # Display counts of gender


    # Display earliest, most recent, and most common year of birth


    print("\nThis took %s seconds." % (time.time() - start_time))
    print('-'*40)


In [ ]:
def main():
    while True:
        city, month, day = get_filters()

        df = load_data(city, month, day)

        time_stats(df)
        station_stats(df)
        trip_duration_stats(df)
        user_stats(df)

        restart = input('\nWould you like to restart? Enter yes or no.\n')
        if restart.lower() != 'yes':
            break



if __name__ == "__main__":
	main()


Hello! Let's explore some US bikeshare data!
